Hybrid Search Langchain

In [3]:
import os 
from dotenv import load_dotenv

load_dotenv()

key = os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")

In [ ]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone, ServerlessSpec

index_name = "hybrid-search-langchain-pinecone"
#initialize pinecone client
pc = Pinecone(api_key=key)

#create index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, # dimension of dense vector
        metric='dotproduct', # sparse vlaue support dotproduct
        spec=ServerlessSpec(cloud='aws',region='us-east-1'),
    )

In [ ]:
# Get the Pinecone index object
index = pc.Index(index_name)
index

In [ ]:
## vector embedding & sparse matrix
import os 
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings()
embeddings

In [ ]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [11]:
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans"
]

# tfidf values on these sentences
bm25_encoder.fit(sentences)

## store the values to a json files 
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 2560.10it/s]


In [ ]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)
retriever


In [ ]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans"
    ]
)

In [ ]:
retriever.invoke("What city did i visit last")